Note that before running each section, the kernel must be restarted.

# 4 bins

In [ ]:
%cd ../data/4_bins/4bins

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os

In [ ]:
max_min = lambda x: (x-x.min())/(x.max()-x.min())
cwd = os.getcwd()
# Read data and scale it by its maximum and minimum.
def prepare_data():    
    X = []
    Y = []
    for i in list(itertools.product([str(0),str(1)], [str(0),str(1)],[str(0),str(1)],[str(0)])):        
        path = os.path.join(cwd, ''.join(i))        
        names = os.listdir(path)       
        
        for name in names:            
            X.append(pd.read_csv(os.path.join(path, name), sep='\t',  usecols=[2]).apply(max_min).to_numpy())            
            Y.append(np.array(eval('[' + ','.join(i) + ']')))            
        
    return np.array(X), np.array(Y)

In [ ]:
X, Y = prepare_data()

In [ ]:
# Split data into test set and the rest set, the size of test set is 20% of total data set
# The test set and the rest set are shuffled randomly.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train_d = np.zeros((X_train.shape[0]*2, X_train.shape[1],  X_train.shape[2]))
y_train_d = np.zeros((y_train.shape[0]*2, y_train.shape[1]))
# Here the white noise is added to increase the robustness of the model and increase the size of the training set, 
# which is quantifyed by its standard deviation "scale".
# If scale = 0, there is no white noise.
# After the additional noise is added, the training set is re-scaled by its maximum and minimum.

for id  in range(X_train.shape[0]):
    X_train_d[id] = X_train[id]
    y_train_d[id] = y_train[id]
    X_train_d[id + X_train.shape[0]] = max_min(X_train[id] + np.random.normal(loc=0, scale=0.1, size = (X_train.shape[1], X_train.shape[2])))
    y_train_d[id + X_train.shape[0]] = y_train[id]

X_train = X_train_d
y_train = y_train_d
del X_train_d
del y_train_d

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from keras import models
from keras import layers
from keras.optimizers import RMSprop
import numpy as np
seed = 7
np.random.seed(seed)

import tensorflow as tf
tf.random.set_seed(seed)

In [ ]:
from keras import backend as K

# Some memory clean-up
K.clear_session()

In [ ]:
# Deep learning model layers
def bulid_model_CNN():  
    
    model_list = []   
    
    cnn_out_1 = 20 #16
    cnn_len_1 = 460 #20
    
    model_list.append(
            layers.Conv1D(cnn_out_1, cnn_len_1, input_shape=(X_train.shape[1],1)),
            ) 
    model_list.append(layers.BatchNormalization())
    model_list.append(layers.Activation('relu'))
    model_list.append(layers.MaxPooling1D(3))    
   
    rnn_out = 16  
    lr_rate = 1e-3
    
    
    model_list.append(layers.Bidirectional(layers.LSTM(units=rnn_out, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)))
    model_list.append(layers.BatchNormalization())
    
    model_list.append(layers.Dense(y_train.shape[1], activation='sigmoid'))
    model = models.Sequential(model_list)   
    
    
    opt = RMSprop(lr=lr_rate)
    model.compile(optimizer=opt, loss='mse')
   
    return model

In [ ]:
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

In [ ]:
%cd ../weights

In [ ]:
# These callback function to schedule the learning rate and save weights of the model during training.
filepath = "weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

callbacks_list = [ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10), 
                   ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='min')  ]

In [ ]:
%%time
k = 4  # k-fold cross validation
num_val_samples = X_train.shape[0] //k
num_epochs = 75
all_scores = []
all_scores_history = []
train_loss_history_list = []

for i in range(k):
    print('processing fold #', i)
    K.clear_session()  # clear previous wieghts to avoid cross-talk
    # Prepare the validation data: data from partition # k
    val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [X_train[:i * num_val_samples],
         X_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)              
    model = bulid_model_CNN()

    history = model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=64, verbose=0, validation_data=(val_data, val_targets),
                        callbacks=callbacks_list)
        
    # Evaluate the model on the validation data
    mae_history = history.history['val_loss']
    train_loss_history = history.history['loss']    

    all_scores_history.append(mae_history)
    train_loss_history_list.append(train_loss_history)        

In [ ]:
averaged_history_val_loss = np.mean(np.array(all_scores_history[1:]), axis=0)
averaged_history_train_loss = np.mean(np.array(train_loss_history_list[1:]), axis=0)
font_size = 16
# plot loss on training set and validation set 
plt.figure(figsize=(8, 6))
plt.plot(averaged_history_train_loss)
plt.plot(averaged_history_val_loss)
plt.title('Model loss', fontdict={'family' : 'Times New Roman', 'size':font_size})
plt.ylabel('Loss', fontdict={'family' : 'Times New Roman', 'size':font_size})
plt.xlabel('Epoch', fontdict={'family' : 'Times New Roman', 'size':font_size})
plt.legend(['Train', 'Validation'], loc='upper right', prop={'family':'Times New Roman', 'size':font_size})
plt.xticks(fontproperties = 'Times New Roman', fontsize=font_size)
plt.yticks(fontproperties = 'Times New Roman', fontsize=font_size)
#plt.savefig('loss.eps',dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
del model

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 16

In [ ]:
def confusion_matrix(pres, y_test):
    cm = np.zeros((8,8))
    for id, pre in enumerate(pres):
        column = int(pre[0]*4 + pre[1] * 2 + pre[2] * 1 )
        row = int(y_test[id, 0]*4 + y_test[id, 1] * 2 + y_test[id, 2] * 1)
        cm[row, column] += 1
    return cm

def heatmap(ma, filename):
    labels = ["000", "001","010","011","100","101","110","111"]
    acc = np.round(np.sum(np.diag(ma))/np.sum(ma), 5)
    ma = pd.DataFrame(ma, index=labels, columns=labels)
    f,ax = plt.subplots(figsize=(9, 6))
    ax = sns.heatmap(ma, annot=True, center=11, cmap='RdYlBu')
    plt.xlabel("Predicted labels",fontdict={'size':16})
    plt.ylabel("Target labels",fontdict={'size':16})
    plt.title('Accuracy = %s %%'%str(acc*100))
    #plt.savefig('%s.eps'%str(filename),dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
from keras.models import load_model

In [ ]:
# The model_names may be different, please check the data/4_bins/weights/ folder
models_names = ['weights-improvement-01-0.17.hdf5', 'weights-improvement-17-0.02.hdf5', 'weights-improvement-30-0.01.hdf5']
for id, name in  enumerate(models_names):
    model = load_model(name)
    pre = model.predict(X_test)
    cm = confusion_matrix(np.around(pre), y_test)
    heatmap(cm, id)

# 20 bins

## Deep learning

Note that before running each section, the kernel must be restarted.

In [ ]:
%cd ../data/20_bins/20bins

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os

In [ ]:
max_min = lambda x: (x-x.min())/(x.max()-x.min())
cwd = os.getcwd()
def prepare_data():    
    X = []
    Y = []
    for i in list(itertools.product([str(0),str(1)], [str(0),str(1)],[str(0),str(1)])):        
        path = os.path.join(cwd, ''.join(i))        
        names = os.listdir(path)       
        
        for name in names:            
            X.append(pd.read_csv(os.path.join(path, name), sep='\t',  usecols=[2]).apply(max_min).to_numpy())            
            Y.append(np.array(eval('[' + ','.join(i) + ']')))            
        
    return np.array(X), np.array(Y)

In [ ]:
X, Y = prepare_data()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train_d = np.zeros((X_train.shape[0]*2, X_train.shape[1],  X_train.shape[2]))
y_train_d = np.zeros((y_train.shape[0]*2, y_train.shape[1]))

for id  in range(X_train.shape[0]):
    X_train_d[id] = X_train[id]
    y_train_d[id] = y_train[id]
    X_train_d[id + X_train.shape[0]] = max_min(X_train[id] + np.random.normal(loc=0, scale=0.1, size = (X_train.shape[1], X_train.shape[2])))
    y_train_d[id + X_train.shape[0]] = y_train[id]

X_train = X_train_d
y_train = y_train_d
del X_train_d
del y_train_d

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from keras import models
from keras import layers
from keras.optimizers import RMSprop
import numpy as np
seed = 7
np.random.seed(seed)

import tensorflow as tf
tf.random.set_seed(seed)

In [ ]:
from keras import backend as K

# Some memory clean-up
K.clear_session()

In [ ]:
def bulid_model_CNN():  
    
    model_list = []   
    
    cnn_out_1 = 20 #16
    cnn_len_1 = 460 #20
    
    model_list.append(
            layers.Conv1D(cnn_out_1, cnn_len_1, input_shape=(X_train.shape[1],1)),
            ) 
    model_list.append(layers.BatchNormalization())
    model_list.append(layers.Activation('relu'))
    model_list.append(layers.MaxPooling1D(3))      
    
    rnn_out = 16  
    lr_rate = 1e-3
    
    
    model_list.append(layers.Bidirectional(layers.LSTM(units=rnn_out, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)))
    model_list.append(layers.BatchNormalization())
    
    model_list.append(layers.Dense(y_train.shape[1], activation='sigmoid'))
    model = models.Sequential(model_list)   
    
    
    opt = RMSprop(lr=lr_rate)
    model.compile(optimizer=opt, loss='mse')
   
    return model

In [ ]:
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

In [ ]:
%cd ../weights

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

callbacks_list = [ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10),
                   ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='min')  ]

In [ ]:
k = 4
num_val_samples = X_train.shape[0] //k
num_epochs = 75
all_scores = []
all_scores_history = []
train_loss_history_list = []

for i in range(k):
    print('processing fold #', i)
    K.clear_session()  # memory clean 
    # Prepare the validation data: data from partition # k
    val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [X_train[:i * num_val_samples],
         X_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)              
    model = bulid_model_CNN()

    history = model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=64, verbose=0, validation_data=(val_data, val_targets),
                        callbacks=callbacks_list)
        
    # Evaluate the model on the validation data
    mae_history = history.history['val_loss']
    train_loss_history = history.history['loss']    

    all_scores_history.append(mae_history)
    train_loss_history_list.append(train_loss_history)    

In [ ]:
averaged_history_val_loss = np.mean(np.array(all_scores_history[1:]), axis=0)
averaged_history_train_loss = np.mean(np.array(train_loss_history_list[1:]), axis=0)
font_size = 16

# plot loss on training set and validation set 
plt.figure(figsize=(8, 6))
plt.plot(averaged_history_train_loss)
plt.plot(averaged_history_val_loss)
plt.title('Model loss', fontdict={'family' : 'Times New Roman', 'size':font_size})
plt.ylabel('Loss', fontdict={'family' : 'Times New Roman', 'size':font_size})
plt.xlabel('Epoch', fontdict={'family' : 'Times New Roman', 'size':font_size})
plt.legend(['Train', 'Test'], loc='upper right', prop={'family':'Times New Roman', 'size':font_size})
plt.xticks(fontproperties = 'Times New Roman', fontsize=font_size)
plt.yticks(fontproperties = 'Times New Roman', fontsize=font_size)
#plt.savefig('loss.eps',dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 16

In [ ]:
def confusion_matrix(pres, y_test):
    cm = np.zeros((8,8))
    for id, pre in enumerate(pres):
        column = int(pre[0]*4 + pre[1] * 2 + pre[2] * 1 )
        row = int(y_test[id, 0]*4 + y_test[id, 1] * 2 + y_test[id, 2] * 1)
        cm[row, column] += 1
    return cm

def heatmap(ma, filename):
    labels = ["000", "001","010","011","100","101","110","111"]
    acc = np.round(np.sum(np.diag(ma))/np.sum(ma), 5)
    ma = pd.DataFrame(ma, index=labels, columns=labels)
    f,ax = plt.subplots(figsize=(9, 6))
    ax = sns.heatmap(ma, annot=True, center=11, cmap='RdYlBu')
    plt.xlabel("Predicted labels",fontdict={'size':16})
    plt.ylabel("Target labels",fontdict={'size':16})
    plt.title('Accuracy = %s %%'%str(acc*100))
    #plt.savefig('%s.eps'%str(filename),dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
from keras.models import load_model

In [ ]:
del model

In [ ]:
models_names = ['weights-improvement-29-0.04.hdf5','weights-improvement-58-0.03.hdf5','weights-improvement-64-0.02.hdf5']
for id, name in  enumerate(models_names):
    model = load_model(name)
    pre = model.predict(X_test)
    cm = confusion_matrix(np.around(pre), y_test)
    heatmap(cm, id)

## Master equation

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 16

In [ ]:
def heatmap(ma, filename):
    labels = ["000", "001","010","011","100","101","110","111"]
    acc = np.round(np.sum(np.diag(ma))/np.sum(ma), 5)
    ma = pd.DataFrame(ma, index=labels, columns=labels)
    f,ax = plt.subplots(figsize=(9, 6))
    ax = sns.heatmap(ma, annot=True, center=11, cmap='RdYlBu')
    plt.xlabel("Predicted labels",fontdict={'size':16})
    plt.ylabel("Target labels",fontdict={'size':16})
    plt.title('Accuracy = %s %%'%str(acc*100))
    #plt.savefig('%s.eps'%str(filename),dpi=300, bbox_inches="tight")
    plt.show()

The data below come from the results of Mathematica notebook "/notebook/Fig5/20bins_fitting.nb"

In [ ]:
cm_master=np.concatenate([np.histogram([0,0,0,0,0,0,0,6,0,0,0,0,0,0,7,0,0,0,0,0],bins=np.arange(9))[0],
np.histogram([0,0,0,1,4,4,4,0,0,0,0,1,4,0,0,0,0,0,0,7],bins=np.arange(9))[0],
np.histogram([0,0,6,2,2,0,2,0,2,0,2,1,0,2,2,2,0,2,0,2],bins=np.arange(9))[0],
np.histogram([0,0,0,0,1,0,0,5,2,0,0,0,0,0,0,0,0,0,0,0],bins=np.arange(9))[0],
np.histogram([1,6,6,0,6,1,1,1,0,1,1,6,1,0,0,0,6,0,5,0],bins=np.arange(9))[0],
np.histogram([0,0,0,0,5,5,2,0,0,0,0,0,2,0,0,2,2,2,0,2],bins=np.arange(9))[0],
np.histogram([3,0,3,3,0,3,3,4,1,4,3,0,2,3,4,0,0,0,3,3],bins=np.arange(9))[0],
np.histogram([0,0,0,0,7,0,0,0,3,0,5,0,0,0,0,0,0,0,1,0],bins=np.arange(9))[0]],axis=0).reshape(8,8)
heatmap(cm_master,'20bins')

# 200 Hz

## Deep learning

Note that before running each section, the kernel must be restarted.

In [ ]:
%cd ../data/resolution/resolution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os

In [ ]:
max_min = lambda x: (x-x.min())/(x.max()-x.min())
cwd = os.getcwd()

def prepare_data():    
    X = []
    Y = []
    for i in list(itertools.product([str(0),str(1)], [str(0),str(1)],[str(0),str(1)])):        
        path = os.path.join(cwd, ''.join(i))        
        names = os.listdir(path)       
        
        for name in names:            
            X.append(pd.read_csv(os.path.join(path, name), sep='\t',  usecols=[2]).apply(max_min).to_numpy())            
            Y.append(np.array(eval('[' + ','.join(i) + ']')))            
        
    return np.array(X), np.array(Y)

In [ ]:
X, Y = prepare_data()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train_d = np.zeros((X_train.shape[0]*2, X_train.shape[1],  X_train.shape[2]))
y_train_d = np.zeros((y_train.shape[0]*2, y_train.shape[1]))

for id  in range(X_train.shape[0]):
    X_train_d[id] = X_train[id]
    y_train_d[id] = y_train[id]
    X_train_d[id + X_train.shape[0]] = max_min(X_train[id] + np.random.normal(loc=0, scale=0.5, size = (X_train.shape[1], X_train.shape[2])))
    y_train_d[id + X_train.shape[0]] = y_train[id]

X_train = X_train_d
y_train = y_train_d
del X_train_d
del y_train_d

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from keras import models
from keras import layers
from keras.optimizers import RMSprop
import numpy as np
seed = 7
np.random.seed(seed)

import tensorflow as tf
tf.random.set_seed(seed)

In [ ]:
from keras import backend as K

# Some memory clean-up
K.clear_session()

In [ ]:
def bulid_model_CNN():  
    
    model_list = []   
    
    cnn_out_1 = 20 #16
    cnn_len_1 = 460 #20
    
    model_list.append(
            layers.Conv1D(cnn_out_1, cnn_len_1, input_shape=(X_train.shape[1],1)),
            ) 
    model_list.append(layers.BatchNormalization())
    model_list.append(layers.Activation('relu'))
    model_list.append(layers.MaxPooling1D(3))  
        
    rnn_out = 16  
    lr_rate = 1e-3
    
    
    model_list.append(layers.Bidirectional(layers.LSTM(units=rnn_out, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)))
    model_list.append(layers.BatchNormalization())
    
    model_list.append(layers.Dense(y_train.shape[1], activation='sigmoid'))
    model = models.Sequential(model_list)   
    
    
    opt = RMSprop(lr=lr_rate)
    model.compile(optimizer=opt, loss='mse')
   
    return model

In [ ]:
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

In [ ]:
%cd ../weights

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

callbacks_list = [ ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10), 
                  ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='min') ] 

In [ ]:
k = 4
num_val_samples = X_train.shape[0] //k
num_epochs = 75
all_scores = []
all_scores_history = []
train_loss_history_list = []

for i in range(k):
    print('processing fold #', i)
    K.clear_session()  # memory clean
    # Prepare the validation data: data from partition # k
    val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [X_train[:i * num_val_samples],
         X_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)              
    model = bulid_model_CNN()

    history = model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=64, verbose=0, validation_data=(val_data, val_targets), callbacks=callbacks_list)
        
    # Evaluate the model on the validation data
    mae_history = history.history['val_loss']
    train_loss_history = history.history['loss']    

    all_scores_history.append(mae_history)
    train_loss_history_list.append(train_loss_history)   

In [ ]:
averaged_history_val_loss = np.mean(np.array(all_scores_history[1:]), axis=0)
averaged_history_train_loss = np.mean(np.array(train_loss_history_list[1:]), axis=0)
font_size = 16

# plot loss on training set and validation set 
plt.figure(figsize=(8, 6))
plt.plot(averaged_history_train_loss)
plt.plot(averaged_history_val_loss)
plt.title('Model loss', fontdict={'family' : 'Times New Roman', 'size':font_size})
plt.ylabel('Loss', fontdict={'family' : 'Times New Roman', 'size':font_size})
plt.xlabel('Epoch', fontdict={'family' : 'Times New Roman', 'size':font_size})
plt.legend(['Train', 'Test'], loc='upper right', prop={'family':'Times New Roman', 'size':font_size})
plt.xticks(fontproperties = 'Times New Roman', fontsize=font_size)
plt.yticks(fontproperties = 'Times New Roman', fontsize=font_size)
#plt.savefig('loss.eps',dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 16

In [ ]:
def confusion_matrix(pres, y_test):
    cm = np.zeros((8,8))
    for id, pre in enumerate(pres):
        column = int(pre[0]*4 + pre[1] * 2 + pre[2] * 1 )
        row = int(y_test[id, 0]*4 + y_test[id, 1] * 2 + y_test[id, 2] * 1)
        cm[row, column] += 1
    return cm

def heatmap(ma, filename):
    labels = ["000", "001","010","011","100","101","110","111"]
    acc = np.round(np.sum(np.diag(ma))/np.sum(ma), 5)
    ma = pd.DataFrame(ma, index=labels, columns=labels)
    f,ax = plt.subplots(figsize=(9, 6))
    ax = sns.heatmap(ma, annot=True, center=11, cmap='RdYlBu')
    plt.xlabel("Predicted labels",fontdict={'size':16})
    plt.ylabel("Target labels",fontdict={'size':16})
    plt.title('Accuracy = %s %%'%str(acc*100))
    #plt.savefig('%s.eps'%str(filename),dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
from keras.models import load_model

In [ ]:
models_names = ['weights-improvement-13-0.11.hdf5', 'weights-improvement-57-0.02.hdf5', 'weights-improvement-100-0.03.hdf5']
for id, name in  enumerate(models_names):
    model = load_model(name)
    pre = model.predict(X_test)
    cm = confusion_matrix(np.around(pre), y_test)
    heatmap(cm, id)

## Mster equation

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 16

In [ ]:
def confusion_matrix(pres, y_test):
    cm = np.zeros((8,8))
    for id, pre in enumerate(pres):
        column = int(pre[0]*4 + pre[1] * 2 + pre[2] * 1 )
        row = int(y_test[id, 0]*4 + y_test[id, 1] * 2 + y_test[id, 2] * 1)
        cm[row, column] += 1
    return cm

def heatmap(ma, filename):
    labels = ["000", "001","010","011","100","101","110","111"]
    acc = np.round(np.sum(np.diag(ma))/np.sum(ma), 5)
    ma = pd.DataFrame(ma, index=labels, columns=labels)
    f,ax = plt.subplots(figsize=(9, 6))
    ax = sns.heatmap(ma, annot=True, center=11, cmap='RdYlBu')
    plt.xlabel("Predicted labels",fontdict={'size':16})
    plt.ylabel("Target labels",fontdict={'size':16})
    plt.title('Accuracy = %s %%'%str(acc*100))
    #plt.savefig('%s.eps'%str(filename),dpi=300, bbox_inches="tight")
    plt.show()

The data below come from the results of Mathematica notebook "/notebook/Fig5/200MHz_fitting.nb"

In [ ]:
cm_master=np.concatenate([np.histogram([0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0],bins=np.arange(9))[0],
np.histogram([0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 5, 1, 1, 1, 1],bins=np.arange(9))[0],
np.histogram([6, 2, 2, 2, 2, 2, 1, 4, 4, 2, 2, 2, 2, 4, 2, 4, 2, 2, 2, 2],bins=np.arange(9))[0],
np.histogram([7, 7, 3, 7, 7, 0, 3, 7, 0, 3, 3, 7, 3, 3, 3, 7, 3, 3, 3, 3],bins=np.arange(9))[0],
np.histogram([0, 4, 0, 0, 4, 4, 4, 4, 0, 4, 4, 0, 0, 4, 4, 4, 4, 4, 4, 4],bins=np.arange(9))[0],
np.histogram([0, 5, 5, 1, 1, 0, 1, 1, 1, 1, 5, 3, 0, 0, 0, 0, 3, 1, 0, 5],bins=np.arange(9))[0],
np.histogram([6, 6, 0, 0, 6, 6, 0, 6, 0, 6, 4, 6, 2, 6, 6, 6, 0, 6, 6, 7],bins=np.arange(9))[0],
np.histogram([3, 7, 0, 7, 7, 0, 6, 7, 7, 3, 0, 0, 0, 0, 7, 2, 7, 1, 7, 0],bins=np.arange(9))[0]],axis=0).reshape(8,8)
heatmap(cm_master,'200Hz')

# Data for additional noise

Note that before running each section, the kernel must be restarted.

The code in this section is to prepare the data with white noise for the master equation fitting in `4bins_fitting.nb`. 

In [ ]:
%cd ..\data\additional_noise\additional_noise

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools

import os
import numpy as np
seed = 7
np.random.seed(seed)
scale_list = np.arange(0.05, 5.5, 0.1)

In [ ]:
X = []
Y = []
cwd = os.getcwd()
max_min = lambda x: (x-x.min())/(x.max()-x.min())

for i in list(itertools.product([str(0),str(1)], [str(0),str(1)],[str(0),str(1)],str(0))):        
    path = os.path.join(cwd, ''.join(i))        
    names = os.listdir(path)       

    for name in names: 
        t = pd.read_csv(os.path.join(path, name), sep='\t',  usecols=[0]).to_numpy()
        X = (pd.read_csv(os.path.join(path, name), sep='\t',  usecols=[2]).apply(max_min).to_numpy())            
        Y.append(np.array(eval('[' + ','.join(i) + ']')))   
        
        for scale in scale_list:
            X_new = np.zeros_like(X)
            X_new = (X + np.random.normal(loc=0, scale=scale, size = X.shape))
                        
            try:
                os.mkdir(''.join(i)+'/'+"{0:.2f}".format(scale))

            except FileExistsError:
                pass
            
            np.savetxt(''.join(i)+'/'+"{0:.2f}".format(scale)+'/'+str(name)[:-4]+'.txt', np.c_[t ,max_min(X_new)] ,delimiter='\t', fmt='%.4e')

In [ ]:
# codes to delete the signals with white noise, usually donot need to run.
"""
import shutil

cwd = os.getcwd()
for i in list(itertools.product([str(0),str(1)], [str(0),str(1)],[str(0),str(1)],str(0))):   
    path = os.path.join(cwd, ''.join(i))  
    for scale in scale_list:
        path1 = os.path.join(path, "{0:.2f}".format(scale))
        try:
            shutil.rmtree(path1)
        except (FileNotFoundError, PermissionError) as e:
            print("Error: %s : %s" % (path1, e.strerror))
"""

# Additional Noises for 4 bins

Note that before running each section, the kernel must be restarted.

In [ ]:
%cd ../data/4_bins/4bins

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 24
import pandas as pd
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os

from keras import models
from keras import layers
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
callbacks_list = [ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)] 
from keras import backend as K

# Some memory clean-up
K.clear_session()

seed = 7
np.random.seed(seed)

import tensorflow as tf
tf.random.set_seed(seed)

In [ ]:
train_scale_list = np.arange(0.0, 3.5, 0.1) # The standard deviation of white noise in the training set
model_list = []
max_min = lambda x: (x-x.min())/(x.max()-x.min())
cwd = os.getcwd()

def prepare_data():    
    X = []
    Y = []
    for i in list(itertools.product([str(0),str(1)], [str(0),str(1)],[str(0),str(1)],[str(0)])):        
        path = os.path.join(cwd, ''.join(i))        
        names = os.listdir(path)       
        
        for name in names:            
            X.append(pd.read_csv(os.path.join(path, name), sep='\t',  usecols=[2]).apply(max_min).to_numpy())            
            Y.append(np.array(eval('[' + ','.join(i) + ']')))            
        
    return np.array(X), np.array(Y)[:,:3]

def bulid_model_CNN():  
    
    model_list = []   
    
    cnn_out_1 = 20 #16
    cnn_len_1 = 460 #20
    
    model_list.append(
            layers.Conv1D(cnn_out_1, cnn_len_1, input_shape=(X_train.shape[1],1)),
            ) 
    model_list.append(layers.BatchNormalization())
    model_list.append(layers.Activation('relu'))
    model_list.append(layers.MaxPooling1D(3))     
    
    rnn_out = 16  
    lr_rate = 1e-3
    
    
    model_list.append(layers.Bidirectional(layers.LSTM(units=rnn_out, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)))
    model_list.append(layers.BatchNormalization())
    
    model_list.append(layers.Dense(y_train.shape[1], activation='sigmoid'))
    model = models.Sequential(model_list)   
    
    
    opt = RMSprop(lr=lr_rate)
    model.compile(optimizer=opt, loss='mse')
   
    return model

In [ ]:
for scale in train_scale_list:
    
    X, Y = prepare_data()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=7)

    X_train_d = np.zeros((X_train.shape[0]*2, X_train.shape[1],  X_train.shape[2]))
    y_train_d = np.zeros((y_train.shape[0]*2, y_train.shape[1]))

    for id  in range(X_train.shape[0]):
        X_train_d[id] = X_train[id]
        y_train_d[id] = y_train[id]
        X_train_d[id + X_train.shape[0]] =(X_train[id] + np.random.normal(loc=0, scale=scale, size = (X_train.shape[1], X_train.shape[2])))
        y_train_d[id + X_train.shape[0]] = y_train[id]

    X_train = X_train_d
    y_train = y_train_d
    del X_train_d
    del y_train_d

    k = 4
    num_val_samples = X_train.shape[0] //k
    num_epochs = 75

    for i in range(k):
        print('processing fold #', i)
        K.clear_session() 
        # Prepare the validation data: data from partition # k
        val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
        val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

        # Prepare the training data: data from all other partitions
        partial_train_data = np.concatenate(
            [X_train[:i * num_val_samples],
             X_train[(i + 1) * num_val_samples:]],
            axis=0)
        partial_train_targets = np.concatenate(
            [y_train[:i * num_val_samples],
             y_train[(i + 1) * num_val_samples:]],
            axis=0)
                   
        model = bulid_model_CNN()

        history = model.fit(partial_train_data, partial_train_targets,
                  epochs=num_epochs, batch_size=64, verbose=0, validation_data=(val_data, val_targets),
                            callbacks=callbacks_list)

    model.save(str(scale))

In [ ]:
scale_list=np.arange(0.0, 5.5, 0.1) # The standard deviation of white noise in the test set
acc_mat = []
for scale_train in train_scale_list:
    acc_list = np.zeros((5, len(scale_list)))
    reconstructed_model = models.load_model(str(scale_train))
    for ii in range(5):

        for ind, scale in enumerate( scale_list):
            X_test_d = np.zeros((X_test.shape[0], X_test.shape[1],  X_test.shape[2]))
            y_test_d = np.zeros((y_test.shape[0], y_test.shape[1]))

            for id  in range(X_test.shape[0]):
                X_test_d[id] = (X_test[id] + np.random.normal(loc=0, scale=scale, size = (X_test.shape[1], X_test.shape[2])))
                y_test_d[id] = y_test[id] 
                
            pre = reconstructed_model.predict(X_test_d)            
            accuracy = sum([1 if np.sum(np.abs(np.around(pre[i]) -y_test_d[i])) == 0 else 0 for i in range(len(pre)) ])/len(pre)
            acc_list[ii,ind] = accuracy

    acc_mat.append(np.mean(np.array(acc_list),axis=0)) 

In [ ]:
plt.plot(acc_mat[0],'-o')

Data is from master equation fitting, which is "calculated by /notebook/4bins_fitting.nb"

The "acc_mat0" is "acc_mat[0]" calculated above. It can be replaced by "acc_mat[0]".

In [ ]:
data = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 0, 3, 4, 4, 4, \
4, 4, 3, 5, 5, 5, 0, 6, 6, 6, 0, 0, 0, 1, 4, 0, 4, 0, 0, 0, 0, 0, 1, \
1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 7, 5, 7, \
3, 4, 3, 7, 7, 0, 6, 0, 7, 6, 4, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, \
2, 2, 2, 3, 3, 5, 3, 7, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 0, 7, 7, 0, 6, \
7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 3, 3, 3, \
3, 3, 4, 4, 4, 4, 4, 2, 5, 5, 0, 5, 6, 0, 7, 7, 6, 7, 7, 7, 7, 1, 0, \
0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 0, 0, 3, 3, 4, 4, 4, 4, \
4, 5, 2, 6, 0, 5, 6, 0, 0, 3, 0, 7, 7, 7, 7, 5, 0, 0, 0, 0, 0, 1, 1, \
6, 1, 0, 2, 2, 2, 0, 2, 3, 5, 3, 3, 3, 0, 0, 4, 0, 4, 0, 2, 6, 2, 4, \
0, 0, 0, 7, 6, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, \
0, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 3, 0, 0, 6, 1, 0, 1, 0, 0, 0, 7, \
7, 7, 7, 7, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 4, 0, 0, 3, \
3, 0, 4, 4, 4, 4, 0, 2, 0, 7, 2, 0, 0, 2, 0, 0, 7, 7, 7, 7, 7, 0, 0, \
0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 0, 0, 0, 0, 3, 4, 0, 4, 4, 4, \
2, 2, 1, 3, 2, 6, 0, 1, 0, 6, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 1, 1, 1, \
1, 0, 2, 2, 2, 5, 0, 4, 0, 4, 3, 0, 0, 4, 4, 0, 4, 2, 2, 0, 0, 2, 0, \
0, 0, 1, 0, 7, 7, 5, 7, 3, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, \
0, 3, 3, 0, 0, 0, 4, 4, 0, 4, 4, 2, 0, 0, 0, 2, 0, 0, 6, 0, 0, 7, 7, \
0, 4, 0, 4, 0, 0, 0, 0, 1, 1, 1, 1, 0, 2, 2, 2, 0, 2, 0, 4, 0, 1, 4, \
6, 0, 0, 4, 4, 7, 0, 2, 0, 2, 6, 0, 7, 4, 6, 0, 0, 7, 7, 7, 0, 0, 0, \
0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 0, 4, 0, 0, 3, 4, 4, 4, 0, 4, 0, \
0, 4, 1, 0, 0, 0, 0, 1, 0, 0, 3, 0, 7, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, \
0, 2, 2, 2, 0, 0, 0, 1, 0, 0, 0, 4, 4, 4, 4, 4, 2, 2, 0, 0, 0, 6, 0, \
0, 0, 0, 1, 1, 0, 7, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 2, 2, 2, 0, 2, \
0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 4, 7, 0, 0, 0, 0, 0, 0, 1, 6, 0, 1, 6, \
0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 0, 0, 4, 0, 3, 0, 0, 0, \
0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 6, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, \
0, 1, 0, 1, 4, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 4, 4, 0, 2, \
0, 0, 0, 0, 0, 0, 0, 6, 7, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0, 3, 1, 0, \
2, 0, 2, 2, 3, 0, 4, 4, 0, 3, 4, 0, 4, 4, 0, 0, 1, 0, 0, 6, 0, 0, 0, \
3, 1, 5, 5, 2, 2, 4, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 0, 2, 2, 0, \
0, 0, 0, 0, 0, 6, 0, 0, 4, 7, 6, 0, 0, 0, 1, 2, 1, 0, 0, 1, 0, 0, 1, \
0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 3, 2, 2, 0, 0, 7, 0, 0, 0, 0, 0, 0, \
0, 0, 4, 0, 2, 0, 0, 0, 6, 0, 1, 0, 6, 0, 1, 6, 2, 0, 0, 4, 0, 4, 0, \
0, 0, 4, 1, 4, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 4, 4, 4, 0, 1, 0, 0, \
0, 0, 0, 1, 0, 0, 6, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, \
0, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 4, 4, 4, 2, 6, 0, 0, 0, 6, 1, 6, 0, \
0, 0, 0, 0, 4, 1, 4, 0, 0, 0, 0, 0, 0, 7, 3, 1, 2, 2, 2, 2, 6, 0, 6, \
0, 4, 4, 0, 0, 4, 0, 4, 2, 0, 0, 0, 0, 6, 0, 0, 0, 6, 0, 0, 0, 4, 0, \
0, 0, 0, 0, 0, 1, 3, 1, 3, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 4, 4, 4, \
0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 4, 0, 0, 1, \
0, 1, 1, 0, 0, 2, 2, 5, 3, 0, 0, 0, 4, 0, 4, 0, 4, 0, 4, 3, 5, 0, 0, \
2, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 4, 1, 1, 0, 2, 1, 0, 2, \
2, 0, 0, 0, 0, 0, 0, 1, 4, 4, 0, 4, 0, 0, 0, 0, 0, 0, 6, 0, 6, 0, 0, \
2, 1, 0, 0, 0, 0, 0, 4, 0, 0, 1, 1, 0, 1, 0, 0, 2, 2, 0, 2, 0, 3, 0, \
3, 0, 0, 0, 0, 0, 4, 0, 0, 1, 0, 1, 6, 0, 0, 0, 6, 2, 0, 5, 0, 0, 0, \
0, 0, 0, 0, 1, 0, 1, 4, 0, 6, 2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 4, 4, 0, \
0, 0, 0, 0, 0, 7, 0, 1, 6, 0, 6, 1, 1, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, \
7, 1, 1, 0, 2, 2, 2, 2, 0, 3, 3, 0, 0, 2, 4, 4, 0, 0, 0, 0, 0, 4, 4, \
6, 0, 6, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 2, 0, 0, \
5, 0, 0, 0, 2, 5, 6, 0, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 0, 3, 0, 0, \
0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 6, 0, 0, 0, 0, 0, 3, 0, \
0, 0, 4, 4, 5, 4, 1, 5, 1, 0, 5, 1, 2, 1, 0, 6, 0, 1, 0, 0, 0, 0, 0, \
0, 0, 0, 0, 1, 1, 3, 1, 2, 2, 0, 0, 4, 5, 0, 3, 0, 3, 0, 0, 4, 4, 4, \
5, 0, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 5, 0, 1, \
0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0, 6, \
0, 0, 6, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 3, 0, \
0, 0, 0, 0, 0, 3, 0, 2, 2, 4, 4, 0, 5, 1, 0, 0, 0, 0, 4, 5, 0, 6, 0, \
0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 1, 4, 0, 1, 0, 2, 0, 4, 0, 0, 7, 0, \
0, 4, 4, 4, 4, 0, 0, 1, 1, 0, 0, 0, 0, 6, 6, 0, 0, 4, 0, 6, 0, 0, 0, \
0, 0, 0, 1, 1, 0, 1, 6, 0, 3, 2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, \
0, 0, 5, 0, 2, 0, 0, 0, 6, 0, 5, 0, 5, 6, 0, 0, 0, 0, 0, 1, 0, 0, 0, \
0, 0, 0, 4, 0, 0, 0, 3, 7, 0, 1, 0, 2, 4, 4, 0, 0, 1, 0, 0, 0, 4, 0, \
4, 0, 2, 0, 4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 2, 1, 1, 0, 2, 0, 0, 0, \
0, 0, 7, 5, 0, 0, 0, 4, 0, 6, 1, 1, 4, 0, 0, 1, 0, 0, 1, 0, 6, 7, 0, \
0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 1, 0, 0, 0, 2, 2, 0, 4, 0, 0, 0, 0, 2, \
0, 4, 0, 0, 0, 0, 3, 0, 0, 2, 1, 6, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 4, \
0, 1, 0, 7, 0, 1, 6, 4, 2, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 4, \
0, 1, 0, 0, 0, 6, 1, 0, 0, 4, 0, 3, 0, 0, 0, 4, 0, 0, 0, 0, 1, 1, 3, \
6, 2, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 4, 1, 0, 0, 5, 0, 6, 0, 0, \
7, 0, 0, 1, 0, 0, 1, 0, 0, 4, 0, 0, 3, 0, 1, 0, 0, 0, 0, 3, 0, 0, 3, \
0, 0, 3, 0, 4, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 6, 1, 0, 6, 4, 0, 0, 0, \
0, 0, 0, 0, 6, 0, 1, 0, 1, 0, 0, 2, 3, 0, 0, 0, 3, 0, 2, 0, 0, 0, 6, \
0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0, 6, 0, 1, 0, 0, 0, 0, 0, \
0, 1, 0, 0, 0, 4, 0, 0, 0, 2, 0, 0, 5, 0, 0, 0, 4, 4, 4, 0, 0, 0, 0, \
0, 0, 4, 0, 0, 6, 0, 0, 2, 0, 0, 0, 4, 3, 0, 4, 0, 0, 0, 1, 1, 1, 2, \
0, 0, 0, 1, 0, 0, 0, 3, 2, 0, 0, 0, 4, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, \
0, 0, 7, 1, 2, 0, 4, 0, 0, 4, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, \
1, 0, 3, 0, 4, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 5, 4, 4, 0, 0, 0, 4, 0, \
2, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 0, 1, 4, 0, \
0, 2, 5, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 1, 0, 0, 0, 0, 4, 4, 0, 0, 0, \
0, 0, 1, 0, 0, 0, 2, 0, 0, 7, 0, 0, 0, 0, 4, 0, 0, 0, 4, 0, 3, 0, 3, \
0, 0, 2, 0, 4, 2, 1, 1, 0, 0, 0, 2, 2, 2, 0, 0, 3, 0, 1, 3, 6, 0, 2, \
2, 2, 2, 0, 0, 0, 3, 0, 0, 4, 2, 0, 0, 2, 6, 0, 2, 0, 2, 0, 0, 0, 0, \
0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 5, 4, 0, 1, 0, 2, 0, 0, 0, 3, 0, 4, 0, \
2, 0, 0, 0, 2, 4, 0, 0, 1, 1, 4, 1, 0, 0, 2, 6, 0, 3, 3, 0, 0, 0, 0, \
0, 0, 4, 0, 2, 0, 2, 6, 1, 0, 0, 0, 0, 5, 0, 2, 0, 0, 0, 0, 0, 0, 0, \
0, 5, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 1, \
0, 1, 0, 3, 0, 0, 2, 0, 4, 3, 0, 4, 0, 0, 4, 0, 4, 4, 0, 1, 5, 0, 0, \
2, 0, 6, 0, 6, 1, 0, 2, 4, 0, 0, 0, 0, 0, 2, 0, 1, 3, 1, 0, 2, 0, 0, \
0, 0, 3, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 5, 1, 5, 0, 4, 0, 0, 0, 1, 0, \
3, 0, 0, 0, 6, 0, 0, 4, 0, 0, 3, 0, 2, 0, 4, 0, 0, 6, 3, 2, 0, 3, 0, \
0, 0, 4, 0, 3, 7, 1, 0, 4, 4, 0, 0, 0, 2, 0, 2, 0, 0, 5, 4, 0, 0, 4, \
0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 4, 0, 6, 0, \
0, 0, 0, 0, 1, 4, 0, 0, 3, 2, 0, 0, 0, 4, 3])

acc_mat0 = np.array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.99473684, 0.99473684,
       0.97368421, 0.98421053, 0.96315789, 0.92631579, 0.91052632,
       0.9       , 0.85263158, 0.85789474, 0.78947368, 0.77894737,
       0.76315789, 0.69473684, 0.68947368, 0.71052632, 0.67894737,
       0.71578947, 0.61578947, 0.65263158, 0.59473684, 0.58421053,
       0.54210526, 0.51578947, 0.56315789, 0.46842105, 0.52631579,
       0.44736842, 0.47894737, 0.46842105, 0.49473684, 0.44210526,
       0.38421053, 0.42631579, 0.39473684, 0.41052632, 0.39473684,
       0.40526316, 0.41052632, 0.33684211, 0.35263158, 0.44210526,
       0.32631579, 0.38947368, 0.31578947, 0.3       , 0.37368421])

acc_list = np.zeros(len(data)//40)
for j in range(len(data)//40):
    acc = []
    for i in range(8):
        acc.append(len(np.where(data[(40*j+5*i): (40*j+5*i+5)]== i)[0]))
    acc_list[j]=sum(acc)

plt.figure(figsize=(12, 8))
plt.plot(np.arange(0.05, 5.5, 0.1), acc_list/40,'-o', label='Master equation')
plt.plot(np.arange(0, 5.5, 0.1), acc_mat0, '-o', label='Deep learning')
plt.xlabel('Test set standard deviation ')
plt.ylabel('Test set accuracy')
plt.legend()
#plt.savefig('testacc.eps',dpi=300, bbox_inches="tight")

In [ ]:
acc_mat = np.squeeze(np.array(acc_mat))

x = np.arange(0, 5.5, 0.1)
y = np.arange(0, 3.5, 0.1)
X, Y = np.meshgrid(x, y)

plt.figure(figsize=(12, 8))
plt.rcParams.update({"font.size":20})
plt.rcParams['font.family'] = 'Times New Roman'
plt.pcolor(X, Y, acc_mat, shading='auto',edgecolors='None',snap=False)
plt.colorbar()
plt.xlabel('Test set standard deviation ')
plt.ylabel('Training set standard deviation')
#plt.savefig('noise_map1.eps')
plt.show()